In [ ]:
# !pip install webdriver-manager
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

In [174]:
# # read more
# for i in range(12):
#     browser.find_element_by_xpath('//*[@id="site-content"]/div/div[2]/div[3]/div/button').click()
#     time.sleep(1)

In [11]:
# pd.set_option('max_colwidth',1000)
# df[['date','link']]

## Crawler

### Open webdriver

In [41]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import time
from selenium import webdriver
import random
import re
import datetime

chromedir = "C:\Program Files (x86)\Google\Chrome\Application"
os.chdir(chromedir)
browser = webdriver.Chrome()
browser.get('https://myaccount.nytimes.com/auth/login?response_type=cookie&client_id=vi')
time.sleep(3)

### Log in

In [43]:
username = 'bj2438@columbia.edu'
browser.find_element_by_id('email').clear()
browser.find_element_by_id('email').send_keys(username)
time.sleep(0.5 + 3*random.random())
browser.find_element_by_xpath('''//*[@id="myAccountAuth"]/div[1]/div/div/form/div/div[2]/button''').click()
time.sleep(0.5 + 3*random.random())
password = '19980526@Columbia'
browser.find_element_by_id('password').clear()
browser.find_element_by_id('password').send_keys(password)
time.sleep(1 + 2*random.random())
browser.find_element_by_xpath('''//*[@id="myAccountAuth"]/div[1]/div/form/div/div[2]/button''').click()
time.sleep(5 + 2*random.random())
print('Note: please type American Airlines into search box')
# ------------------------------------login finished---------------------------------------------
# -----------------------手动搜索----American Airlines-------PepsiCo--------Disney----------------------------

Note: please type American Airlines into search box


### Prepare urls & Save cookies

In [44]:
# specify date range
a=120 # read more times
browser.find_element_by_class_name('css-p5555t').click()
time.sleep(0.5)
browser.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[2]/div/div/div[1]/div/div/div/ul/li[6]/button').click()
time.sleep(0.5)
start = '01/01/2010'
browser.find_element_by_id('startDate').clear()
browser.find_element_by_id('startDate').send_keys(start)
end = '06/30/2021'
browser.find_element_by_id('endDate').clear()
browser.find_element_by_id('endDate').send_keys(end)
print(f'date range set from {start} to {end}, done')

date range set from 01/01/2010 to 06/30/2021, done


In [45]:
# order by newest
browser.find_element_by_class_name('css-v7it2b').click()
browser.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[1]/form/div[2]/div/select/option[1]').click()
print('search order set by relevance, done')
# browser.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/div[1]/form/div[2]/div/select/option[2]').click()
# print('search order set from newest to oldest, done')

search order set by relevance, done


In [46]:
# specify date range
a=120 # read more times
print('start crawling...')

# ------------------------------------ready for crawling------------------------------------------

# Important!
ct = 0
try:
    for i in range(a):
        browser.find_element_by_xpath('//*[@id="site-content"]/div/div[2]/div[3]/div/button').click()
        ct += 1
        if ct%10 == 0:
            print(f'load {ct} / {a} times...')
        time.sleep(2)
except:
    print('All articles loaded, or ERRORs happened!')
print('finish loading!')
    

results_page = BeautifulSoup(browser.page_source,'lxml')

dp = results_page.find_all('div',class_='css-e1lvw9')
date_ls = results_page.find_all('span',class_='css-17ubb9w')
n = len(dp)
date = []
title = []
abstract = []
author = []
link = []
genre = []

for i in range(n):
#     try:
#         date.append(re.search(r'/(\d{4}/\d{2}/\d{2})', dp[i].find('a').get('href')).groups()[0])
#     except:
#         date.append(date_ls[i].get_text())
    original_date = date_ls[i].get_text()
    if ',' not in original_date:
        original_date += ', 2021'
    new_date = original_date.replace('Jan.','01,').replace('Feb.','02,').replace('March','03,').replace('April','04,').replace('May','05,').replace('June','06,').replace('July','07,').replace('Aug.','08,').replace('Sept.','09,').replace('Oct.','10,').replace('Nov.','11,').replace('Dec.','12,').replace(', ','/')
    date.append(datetime.datetime.strftime(datetime.datetime.strptime(new_date,'%m/%d/%Y'),'%Y/%m/%d'))
    title.append(dp[i].find('h4',class_='css-2fgx4k').get_text())
    try:
        abstract.append(dp[i].find('p',class_='css-16nhkrn').get_text())
    except:
        abstract.append('')
    try:
        author.append(dp[i].find('p',class_='css-15w69y9').get_text()[3:])
    except:
        author.append('')
    l = dp[i].find('a').get('href')
    try:
        if l[:6] == 'https:':
            link.append(l)
        else:
            link.append('https://www.nytimes.com' + l)
    except:
        link.append('')
    genre.append(dp[i].find('p',class_='css-myxawk').get_text())
print('basic information gathered')
print(f'actual/maximum articles in the end:  {n}/{a*10+10}')
    
# -----------------------------------------finish crawling------------------------------------------
# Note: There maybe a few missing values for abstract and author, 
#       and date format mostly be like YYYY/MM/DD, few like Sept. 11, 2019 or June 11

df = pd.DataFrame()
df['date'] = np.array(date)
df['title'] = np.array(title)
df['abstract'] = np.array(abstract)
df['author'] = np.array(author)
df['link'] = np.array(link)
df['genre'] = np.array(genre)
df['article'] = ''
df
print('dataframe built')

c = browser.get_cookies()
cookies = {}
for cookie in c:
    cookies[cookie['name']] = cookie['value']
print('cookies done')

start crawling...
load 10 / 120 times...
load 20 / 120 times...
load 30 / 120 times...
load 40 / 120 times...
load 50 / 120 times...
load 60 / 120 times...
load 70 / 120 times...
load 80 / 120 times...
load 90 / 120 times...
load 100 / 120 times...
load 110 / 120 times...
load 120 / 120 times...
finish loading!
basic information gathered
actual/maximum articles in the end:  1010/1210
dataframe built
cookies done


### First trial of getting articles content

In [47]:
# delete null link records
df = df[df['link']!='']

In [48]:
headers = {
    'method': 'GET',
    'path': '/',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',

}

print('gather single article content...')

num_of_success = 0

for k in range(len(df)):
    # 使用该cookie完成请求
    u = df['link'].loc[k]
    response = requests.get(url=u, headers=headers, cookies=cookies)
    time.sleep(0.2)
    results_page = BeautifulSoup(response.content,'lxml')

    if u[24:29] == 'video' or u[24:33] == 'slideshow' or u[24:31] == 'podcast':
        s = 'Not Applicable'
    
    elif u[:16] == 'https://www.nyti':
        paras = results_page.find_all('div',class_='css-53u6y8')
        # paras = paras[:-1]

        # 存在无内容的段落，按''处理
        # 有内容的段落直接连接在一起
        s = ''
        for j in range(len(paras)):
            ls = paras[j].find_all('p')
            try:
                s += paras[j].find('h2', class_ = 'css-1aoo5yy').get_text()
            except:
                pass
            s += ' '
            for i in range(len(ls)):
                s += ls[i].get_text()
                s += ' '

#     elif u[:16] == 'https://intransi' or u[:16] == 'https://dealbook' or u[:16] == 'https://bucks.bl':
    else:
        paras = results_page.find_all('p',class_='story-body-text')
        s = ''
        for j in range(len(paras)):
            s += paras[j].get_text().replace('  ', '').replace('\n',' ').replace('\xa0','')
            
    df['article'].loc[k] = s.strip()
    
    if df['article'].loc[k] != '':
        num_of_success += 1

    if k % 50 == 49 or k+1 == len(df):
        print(f'articles {k+1}/{len(df)}')
    time.sleep(0.2)

print(f'finished the initial trial of getting articles, with {num_of_success}/{n} of success.')

gather single article content...
articles 50/1009
articles 100/1009


KeyError: 102

## Drop duplicates

In [50]:
df['magic'] = df['date']+df['title']
print(f"unique/total: {len(df['magic'].unique())}/{len(df['magic'])}")
df_new = df.drop_duplicates(['magic']).reset_index()
print('duplicates dropped!')
print(f"unique/total: {len(df_new['magic'].unique())}/{len(df_new['magic'])}")
del df_new['magic']## Check the missing articles
df_stored, df = df, df_new

unique/total: 761/1009
duplicates dropped!
unique/total: 761/761


## Check the missing articles

In [51]:
if not c:
    c = browser.get_cookies()
    cookies = {}
    for cookie in c:
        cookies[cookie['name']] = cookie['value']
    print('cookies done')
else:
    print('cookies exists')

    
total_check_times = 10
    
print(f'check the missing articles {total_check_times} times...')
print('start!')

for c in range(total_check_times):
    for k in range(len(df)):
        if df['article'].loc[k] == '':
            u = df['link'].loc[k]
            response = requests.get(url=u, headers=headers, cookies=cookies)
            time.sleep(0.2)
            results_page = BeautifulSoup(response.content,'lxml')
            
            if u[24:29] == 'video' or u[24:33] == 'slideshow' or u[24:31] == 'podcast':
                s = 'Not Applicable'
                
            elif u[:16] == 'https://www.nyti':
                paras = results_page.find_all('div',class_='css-53u6y8')
                # paras = paras[:-1]

                # 存在无内容的段落，按''处理
                # 有内容的段落直接连接在一起
                s = ''
                for j in range(len(paras)):
                    ls = paras[j].find_all('p')
                    try:
                        s += paras[j].find('h2', class_ = 'css-1aoo5yy').get_text()
                    except:
                        pass
                    s += ' '
                    for i in range(len(ls)):
                        s += ls[i].get_text()
                        s += ' '

        #     elif u[:16] == 'https://intransi' or u[:16] == 'https://dealbook' or u[:16] == 'https://bucks.bl':
            else:
                paras = results_page.find_all('p',class_='story-body-text')
                s = ''
                for j in range(len(paras)):
                    s += paras[j].get_text().replace('  ', '').replace('\n',' ').replace('\xa0','')

            df['article'].loc[k] = s.strip()
            if df['article'].loc[k] != '':
                state = 'success!'
            else:
                state = 'fail...'
            print(f'checking round {c+1}:  articles {k+1}/{len(df)}: {state}')
            time.sleep(0.2)
            
    print(f"Round {c+1}/{total_check_times} finished: missing/all articles {len(df[df['article']==''])}/{len(df)}")

cookies exists
check the missing articles 10 times...
start!


C:\Users\woody\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


checking round 1:  articles 57/761: success!
checking round 1:  articles 71/761: success!
checking round 1:  articles 76/761: success!
checking round 1:  articles 98/761: success!
checking round 1:  articles 99/761: success!
checking round 1:  articles 100/761: success!
checking round 1:  articles 101/761: fail...
checking round 1:  articles 102/761: success!
checking round 1:  articles 103/761: success!
checking round 1:  articles 104/761: success!
checking round 1:  articles 105/761: success!
checking round 1:  articles 106/761: success!
checking round 1:  articles 107/761: success!
checking round 1:  articles 108/761: success!
checking round 1:  articles 109/761: success!
checking round 1:  articles 110/761: fail...
checking round 1:  articles 111/761: success!
checking round 1:  articles 112/761: success!
checking round 1:  articles 113/761: success!
checking round 1:  articles 114/761: fail...
checking round 1:  articles 115/761: success!
checking round 1:  articles 116/761: succe

ConnectionError: HTTPSConnectionPool(host='6thfloor.blogs.nytimes.com', port=443): Max retries exceeded with url: /2014/03/11/talking-with-readers-about-owen-disney-and-autism/?searchResultPosition=327 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018809BBAD30>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [65]:
u = 'https://intransit.blogs.nytimes.com/2010/03/10/travel-deals-airtran-and-american-airlines-sales/?searchResultPosition=39'
response = requests.get(url=u, headers=headers, cookies=cookies)
results_page = BeautifulSoup(response.content,'lxml')
paras = results_page.find_all('p',class_='story-body-text')
s = ''
for j in range(len(paras)):
    s += paras[j].get_text().replace('  ', '').replace('\n',' ').replace('\xa0','')

In [22]:
df[['date','title','abstract','author','genre','article']]

,date,title,abstract,author,genre,article
0,2021/05/14,A Tahini Lemon Cranberry Bar Dreams of Whole F...,With workers at home and stores stingy with sh...,Julie Creswell,Business,Lupii bars were supposed to have a breakout ye...
1,2021/02/09,Aunt Jemima Has a New Name After 131 Years: Th...,Quaker Oats announced it would drop the name A...,Neil Vigdor,Business,It has been a staple of American breakfast tab...
2,2011/09/20,PepsiCo to Foster Chickpeas in Ethiopia,The company said an increased yield of the cro...,Stephanie Strom,Business,PepsiCo plans to announce a new venture on Wed...
3,2013/01/25,PepsiCo Will Halt Use of Additive in Gatorade,A petition on Change.org gets results.,Stephanie Strom,Style,PepsiCo announced on Friday that it would no l...
4,2010/03/16,PepsiCo Adds to Buyback Plan,PepsiCo said Monday that it would buy back up ...,Dealbook,Business,PepsiCo said Monday that it would buy back up ...
...,...,...,...,...,...,...
734,2020/09/29,"Corrections: Sept. 30, 2020",Corrections that appeared in print on Wednesda...,,Corrections,NATIONAL An article on Monday about President ...
735,2014/01/06,F.D.A. Must Act on Food,Michael F. Jacobson of the Center for Science ...,,Opinion,To the Editor: “Social Media as a Megaphone t...
736,2016/10/17,"Coke, Pepsi and Climate Change",A U.S. senator says the lobbying presence in C...,,Opinion,"To the Editor: Re “Soda Giants Back, Then Lobb..."
737,2017/08/04,"Corrections: August 5, 2017","Corrections appearing in print on Saturday, Au...",,Corrections,BUSINESS DAY An article on Thursday about a ne...


In [140]:
len(df[df['article']==''])

29

In [156]:
df[['date','title']].nunique()

date     677
title    812
dtype: int64

In [170]:
len(df_stored[df_stored['article']==''])

29

In [ ]:
df.to_excel(r'C:\Users\woody\iCloudDrive\First-year MSOR\MS Columbia 2nd data challenge\NYT_Disney.xlsx')